In [1]:
import ipywidgets as widgets
from IPython.display import display

In [2]:
stations="""07005	ABBEVILLE
07015	LILLE-LESQUIN
07020	PTE DE LA HAGUE
07027	CAEN-CARPIQUET
07037	ROUEN-BOOS
07072	REIMS-PRUNAY
07110	BREST-GUIPAVAS
07117	PLOUMANAC'H
07130	RENNES-ST JACQUES
07139	ALENCON
07149	ORLY
07168	TROYES-BARBEREY
07181	NANCY-OCHEY
07190	STRASBOURG-ENTZHEIM
07207	BELLE ILE-LE TALUT
07222	NANTES-BOUGUENAIS
07240	TOURS
07255	BOURGES
07280	DIJON-LONGVIC
07299	BALE-MULHOUSE
07314	PTE DE CHASSIRON
07335	POITIERS-BIARD
07434	LIMOGES-BELLEGARDE
07460	CLERMONT-FD
07471	LE PUY-LOUDES
07481	LYON-ST EXUPERY
07510	BORDEAUX-MERIGNAC
07535	GOURDON
07558	MILLAU
07577	MONTELIMAR
07591	EMBRUN
07607	MONT-DE-MARSAN
07621	TARBES-OSSUN
07627	ST GIRONS
07630	TOULOUSE-BLAGNAC
07643	MONTPELLIER
07650	MARIGNANE
07661	CAP CEPET
07690	NICE
07747	PERPIGNAN
07761	AJACCIO
07790	BASTIA
61968	GLORIEUSES
61970	JUAN DE NOVA
61972	EUROPA
61976	TROMELIN
61980	GILLOT-AEROPORT
61996	NOUVELLE AMSTERDAM
61997	CROZET
61998	KERGUELEN
67005	PAMANDZI
71805	ST-PIERRE
78890	LA DESIRADE METEO
78894	ST-BARTHELEMY METEO
78897	LE RAIZET AERO
78922	TRINITE-CARAVEL
78925	LAMENTIN-AERO
81401	SAINT LAURENT
81405	CAYENNE-MATOURY
81408	SAINT GEORGES
81415	MARIPASOULA
89642	DUMONT D'URVILLE"""

name_to_code={}
for t in stations.split('\n'):
    code, name= t.split('\t')
    name_to_code[name]=int(code)

In [3]:
gui_station = widgets.Dropdown(
    options=name_to_code.keys(),
    value='ORLY',
    description='Station:',
)

In [4]:
desc_to_columns={"Vitesse du vent moyen 10 mn (m/s)":'ff',
        "Température (°C)":'t',
        "Humidité (%)":'u',
        "Visibilité horizontale (m)" : 'vv',
        "Nebulosité totale (%)" : 'n',
        "Pression station (Pa)" : 'pres',
        "Hauteur totale de la couche de neige, glace, autre (m)": 'ht_neige',
        "Précipitation dans la dernière heure (mm)" : 'rr1'}

In [5]:
gui_mesure = widgets.Dropdown(
    options=desc_to_columns.keys(),
    value="Température (°C)",
    description='Mesure:',
)

In [6]:
import pandas as pd

In [7]:
import seaborn as sns; sns.set(rc={'figure.figsize':(20,10)})

In [8]:
df=pd.read_csv("../../data/Meteo/meteo_france.csv.zip",parse_dates=['date'])

In [ ]:
df['t']=df['t']-273.15 # K → C

In [9]:
gui_year = widgets.Dropdown(
    options=df['date'].dt.year.astype('str').astype('object').unique(),
    value='2019',
    description='Année:',
)

In [10]:
import matplotlib.pyplot as plt
import matplotlib.dates as md
from IPython.display import clear_output

In [12]:
out=widgets.Output()
def do_plot(unused):
    ax=sns.lineplot(data=df.loc[df['numer_sta']==name_to_code[gui_station.value],].set_index('date').loc[gui_year.value],
                x='date', y=desc_to_columns[gui_mesure.value])
    ax.xaxis.set_major_locator(md.WeekdayLocator(byweekday = 1))
    ax.xaxis.set_minor_locator(md.DayLocator())
    ax.xaxis.set_major_formatter(md.DateFormatter('%Y-%m-%d'))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation = 90)
    ax.xaxis.set_minor_locator(md.DayLocator(interval = 1))
    ax.set_xlabel('Date')
    ax.set_ylabel(gui_mesure.value)
    ax.set_title("Relevés à "+gui_station.value+" en "+gui_year.value)
    with out :
        clear_output(wait=True)
        plt.show()

In [13]:
gui_station.observe(do_plot)
gui_mesure.observe(do_plot)
gui_year.observe(do_plot)
display(gui_station, gui_mesure, gui_year)

Dropdown(description='Station:', index=10, options=('ABBEVILLE', 'LILLE-LESQUIN', 'PTE DE LA HAGUE', 'CAEN-CAR…

Dropdown(description='Mesure:', index=1, options=('Vitesse du vent moyen 10 mn (m/s)', 'Température (°C)', 'Hu…

Dropdown(description='Année:', index=22, options=('1997', '1998', '1999', '2000', '2001', '2002', '2003', '200…

In [14]:
do_plot(None)
display(out)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 1440x720 with 1 Axes>', '…